In [1]:
import pandas as pd

In [2]:
file = "./data/nmhss/nmhss-puf-2019-csv.csv"
nm2019 = pd.read_csv(file)
nm2019.head()

,CASEID,LST,MHINTAKE,MHDIAGEVAL,MHREFERRAL,SMISEDSUD,TREATMT,ADMINSERV,SETTINGIP,SETTINGRC,...,LICENMH,LICENSED,LICENPH,LICENSEDFCS,LICENHOS,JCAHO,CARF,COA,CMS,OTHSTATE
0,201900001,AK,1,1,1,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,201900002,AK,1,1,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,201900003,AK,1,1,1,1,1,0,0,0,...,1,1,1,0,0,1,0,0,1,0
3,201900004,AK,1,1,1,1,1,0,0,0,...,1,1,1,0,0,1,0,0,1,0
4,201900005,AK,1,1,1,1,1,1,0,0,...,1,0,0,0,0,0,0,1,1,0


In [3]:
nm2019.shape

(12472, 140)

## Picking only a few relevant columns

Reason. The goal of this portion of the program is to understand the following

a. Number of total facilities per state

b. Number of facilities per state that partake in mental health improvement

c. Number of facilities per state that partake in substance abuse treatment

d. Compare a-c for years 2019 and 2014.


In [4]:
nm2019 = nm2019[[
    'LST',
    'TREATMT',
    'SETTINGIP',
    'SETTINGRC',
    'SETTINGDTPH',
    'SETTINGOP',
    'FACILITYTYPE',
    'FOCUS',
    'MHINTAKE',
    'MHDIAGEVAL',
    'MHREFERRAL',
    'SRVC31'
]]
nm2019.head()

,LST,TREATMT,SETTINGIP,SETTINGRC,SETTINGDTPH,SETTINGOP,FACILITYTYPE,FOCUS,MHINTAKE,MHDIAGEVAL,MHREFERRAL,SRVC31
0,AK,1,0,0,0,1,7,3,1,1,1,1
1,AK,1,0,0,0,1,7,3,1,1,1,1
2,AK,1,0,0,0,1,9,3,1,1,1,1
3,AK,1,0,0,0,1,10,4,1,1,1,0
4,AK,1,0,0,0,1,7,1,1,1,1,1


### Data filtering and derived data
1. Created a derived column "MentalHealthAssistance" <- (if any of the colums  
   SETTINGIP, SETTINGRC, SETTINGDTPH, FOCUS = 1 or 3 , MHHINTAKE, MHDIAGEVAL, MHREFERRAL is set to 1
2. 'TREATMENT' column already indicates those facilities treating substance abuse

In [5]:
def label_mha (row):
    if (row['SETTINGIP'] == 1) or \
       (row['SETTINGRC'] == 1) or \
       (row['SETTINGDTPH'] == 1) or \
       (row['FOCUS'] == 1) or \
       (row['FOCUS'] == 3) or \
       (row['MHINTAKE'] == 1) or \
       (row['MHDIAGEVAL'] == 1) or \
       (row['MHREFERRAL'] == 1):
        return int(1)
    else:
        return 0
        



nm2019['MentalHealthAssistance'] = nm2019.apply (lambda row: label_mha(row), axis=1)
nm2019['MentalHealthAssistance']= nm2019['MentalHealthAssistance'].astype(int)

nm2019['SubstanceAbuseAssistance']= nm2019['TREATMT'].astype(int)
nm2019.head()


,LST,TREATMT,SETTINGIP,SETTINGRC,SETTINGDTPH,SETTINGOP,FACILITYTYPE,FOCUS,MHINTAKE,MHDIAGEVAL,MHREFERRAL,SRVC31,MentalHealthAssistance,SubstanceAbuseAssistance
0,AK,1,0,0,0,1,7,3,1,1,1,1,1,1
1,AK,1,0,0,0,1,7,3,1,1,1,1,1,1
2,AK,1,0,0,0,1,9,3,1,1,1,1,1,1
3,AK,1,0,0,0,1,10,4,1,1,1,0,1,1
4,AK,1,0,0,0,1,7,1,1,1,1,1,1,1


### Grouping and Aggregating
1. Group by state and count the number of mentalhealth and substance abuse assistance 

In [6]:
summary_mh = nm2019[['LST', 'MentalHealthAssistance', 'SubstanceAbuseAssistance']]
summary_mh = summary_mh.groupby(['LST', 'MentalHealthAssistance']).agg(mhCount = ('MentalHealthAssistance', 'count')).reset_index()
summary_mh

,LST,MentalHealthAssistance,mhCount
0,AK,1,93
1,AL,1,180
2,AR,1,202
3,AZ,1,421
4,CA,1,952
5,CO,1,190
6,CT,1,230
7,DC,1,39
8,DE,1,36
9,FL,1,520


In [7]:
import numpy as np
summary_sa = nm2019[['LST', 'MentalHealthAssistance', 'SubstanceAbuseAssistance']]
summary_sa = summary_sa.groupby(['LST', 'SubstanceAbuseAssistance']).agg(saCount = ('SubstanceAbuseAssistance', 'count')).reset_index()
summary_sa["SubstanceAbuseAssistance"] = np.where(summary_sa["SubstanceAbuseAssistance"] == 1, "Y", "N")
summary_sa

,LST,SubstanceAbuseAssistance,saCount
0,AK,N,31
1,AK,Y,62
2,AL,N,113
3,AL,Y,67
4,AR,N,112
...,...,...,...
101,WV,Y,96
102,WY,N,7
103,WY,Y,38
104,ZZ,N,2


### Final data transformation to make the summary look good.
1. Group by state and count the number of mentalhealth and substance abuse assistance 

In [8]:
import numpy as np
#summary_sa["SubstanceAbuseAssistance"] = np.where(summary_sa["SubstanceAbuseAssistance"] == 1, "Y", "N")
summary_mh["MentalHealthAssistance"] = np.where(summary_mh["MentalHealthAssistance"] == 1, "Y", "N")
summary_mh

,LST,MentalHealthAssistance,mhCount
0,AK,Y,93
1,AL,Y,180
2,AR,Y,202
3,AZ,Y,421
4,CA,Y,952
5,CO,Y,190
6,CT,Y,230
7,DC,Y,39
8,DE,Y,36
9,FL,Y,520


### Final summary
Merge the 2 summary tables for mental health and substance abuse

In [14]:
import states

summary = summary_mh.merge(summary_sa, how = "inner", left_on = "LST", right_on = "LST")
summary['State'] = summary[['LST']].apply(states.statelookupbyabbr, axis=1)
len(summary['State'].unique())

53

In [15]:
summary

,LST,MentalHealthAssistance,mhCount,SubstanceAbuseAssistance,saCount,State
0,AK,Y,93,N,31,Alaska
1,AK,Y,93,Y,62,Alaska
2,AL,Y,180,N,113,Alabama
3,AL,Y,180,Y,67,Alabama
4,AR,Y,202,N,112,Arkansas
...,...,...,...,...,...,...
103,WV,Y,117,Y,96,West Virginia
104,WY,Y,45,N,7,Wyoming
105,WY,Y,45,Y,38,Wyoming
106,ZZ,Y,9,N,2,Other
